In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
import pickle
import youtokentome as yttm
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import ComplementNB
import numpy as np
from sklearn import metrics

In [3]:
with open('bow-language-detection.pkl', 'rb') as fopen:
    bow = pickle.load(fopen)

In [4]:
with open('train-test.json') as fopen:
    train_test = json.load(fopen)
    
train_test.keys()

dict_keys(['train_X', 'test_X', 'train_Y', 'test_Y'])

In [5]:
test_Y = LabelEncoder().fit_transform(train_test['test_Y'])

In [6]:
bpe = yttm.BPE(model='language-detection.model')

In [7]:
test_subs = [' '.join(s) for s in bpe.encode(train_test['test_X'], output_type=yttm.OutputType.SUBWORD)]
len(test_subs)

4729650

In [8]:
test_X = bow.transform(test_subs)

In [9]:
import tensorflow as tf

In [10]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [11]:
class Model:
    def __init__(self, learning_rate, dimension = 32, output = 6):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        embeddings = tf.Variable(tf.truncated_normal([400000,dimension]))
        embed = tf.nn.embedding_lookup_sparse(embeddings, self.X, self.W, combiner='mean')
        self.embed = embed
        self.logits = tf.layers.dense(embed, output)

In [12]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(1e-3)
sess.run(tf.global_variables_initializer())

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [13]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'lang-detection-w/model.ckpt')

INFO:tensorflow:Restoring parameters from lang-detection-w/model.ckpt


In [14]:
from tqdm import tqdm

batch_size = 128
predicted_Y = []

for i in tqdm(range(0, test_X.shape[0], batch_size)):
    index = min(i + batch_size, test_X.shape[0])
    batch_x = convert_sparse_matrix_to_sparse_tensor(test_X[i: index])
    results = sess.run(model.logits, feed_dict = {
                model.X: batch_x[0],
                model.W: batch_x[1],
            })
    predicted_Y.extend(np.argmax(results, axis = 1))

100%|██████████| 36951/36951 [01:00<00:00, 612.88it/s]


In [15]:
len(test_Y) == len(predicted_Y)

True

In [16]:
print(
    metrics.classification_report(
        test_Y,
        predicted_Y,
        target_names = ['eng', 'ind', 'malay', 'manglish', 'other', 'rojak'],
        digits = 5
    )
)

              precision    recall  f1-score   support

         eng    0.96760   0.97401   0.97080    553739
         ind    0.97635   0.96131   0.96877    576059
       malay    0.96985   0.98498   0.97736   1800649
    manglish    0.98036   0.96569   0.97297    181442
       other    0.99641   0.99627   0.99634   1428083
       rojak    0.94221   0.84302   0.88986    189678

    accuracy                        0.97779   4729650
   macro avg    0.97213   0.95421   0.96268   4729650
weighted avg    0.97769   0.97779   0.97760   4729650

